First of all, so this to make sure that your device won't crash in middle of trainig

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
import warnings
warnings.filterwarnings('ignore')


Login into huggingface

In [3]:

from huggingface_hub import login
login()

Import the required libraries

In [4]:
# -*- coding: utf-8 -*-
"""
@author: saura
"""

from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, AutoTokenizer

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, LoraConfig, PeftModel


import logging
logger = logging.getLogger(__name__)


tqdm.pandas()




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: C:\Users\saura\.conda\envs\gpt1\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary C:\Users\saura\.conda\envs\gpt1\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


All the variables define them, description is there

In [5]:
bnb_config= BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_use_double_quant=True,
        bnb_8bit_quant_type="nf4",
        bnb_8bit_compute_dtype=torch.bfloat16,
        llm_int8_skip_modules= ['decoder', 'lm_head',  'wo'],
    )

# base model name
model_id = 'meta-llama/Llama-2-7b-hf';


# Fine-tuned model name
new_model = "sqlModel"

# Load the entire model on the GPU 0
device_map = {"": 0}

# output directory for model
output_dir = "trained1"



Get the maximum length of the dataset

In [6]:
def get_max_length():
  max_length = None
  for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
    max_length = getattr(model.config, length_setting, None)
    if max_length:
      logger.info(f"Found max lenth: {max_length}")
      break
  if not max_length:
    max_length = 32_000
    logger.info(f"Using default max length: {max_length}")
  return max_length

Load the model, tokenizer and the dataset

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             trust_remote_code=True, 
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             quantization_config=bnb_config,
                                             low_cpu_mem_usage=True,
                                             load_in_8bit=True,
                                             )

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          use_fast = False,
                                          max_length=get_max_length(),
                                          )



# Step 2: Load the dataset
dataset = load_dataset('b-mc2/sql-create-context', split="train")

dataset


Configurations

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        r=32,
        lora_dropout=0.1,
        
        bias="none",
        task_type="CAUSAL_LM", 
)



# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


max_seq_length = get_max_length() # max sequence length for model and packing of the dataset

args = TrainingArguments(
    output_dir=output_dirs,
    num_train_epochs=3,
    per_device_train_batch_size= 12, #6 if use_flash_attention else 3,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    #optim="paged_adamw_32bit",
    
    #torch_compile=True, # optimizations
    #optim="adamw_torch_fused", # improved optimizer 
    #optim="adamw_bnb_8bit", #     #['adamw_hf', 'adamw_torch', 'adamw_torch_fused', 'adamw_torch_xla', 'adamw_apex_fused', 'adafactor', 'adamw_bnb_8bit', 'adamw_anyprecision', 'sgd', 'adagrad']

    save_steps=50,
    save_strategy="steps",
    logging_steps=10,
    learning_rate=2e-4,
    bf16=True,
    # fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm= False, #True # disable tqdm since with packing values are in correct
)





In [ ]:
print(max_seq_length)

Now this is really important function, need to format the dataset otherwise it won't work

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['context'])):
        print(i)
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

tokenizer.pad_token = tokenizer.eos_token

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [ ]:
# Step 5: Define the Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length, 
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func, 
    data_collator = collator,
    args=args,
)

So, if training for the first time, set resume_from_checkpoint = False, otherwise it'll throw error, and then set it true once it has saved a checkpoint, it'll save time for retraining if the system crashes

In [ ]:
# Step 6: Train the model
trainer.train(resume_from_checkpoint=True)

In [ ]:
# Step 6: Save the model
trainer.save_model(output_dir)

Now reload the model, restart the kernel before doing it, if just done training, then skip the next cell and do the inference directly

In [7]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

model = model.cuda()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Need to make sure both the model and the prompt are loaded in the same device, i.e. gpu here in this case

In [ ]:
print(next(model.parameters()).device)

prompt = "What is a sql query?"  # adjust this based on your requirements
encoded_prompt = tokenizer.encode(prompt, return_tensors="pt")

encoded_prompt = encoded_prompt.cuda()

print(encoded_prompt.device)  # Should output cuda:X

Do the inference

In [ ]:
# Generate a sequence from the provided prompt
output_sequence = model.generate(
    encoded_prompt,
    max_length=100,  # adjust as needed
    temperature=1.0,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
)

generated_sequence = output_sequence[:, len(encoded_prompt[0]):][0]
decoded_sequence = tokenizer.decode(generated_sequence, skip_special_tokens=True)

Print the inference, ie output

In [ ]:
print(decoded_sequence)